# QPM : Assignment 6

By Valentin PICAVET

# Downloading Data

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

In [2]:
list_stock = ["AAPL", "MSFT", "AMZN", "NVDA", "TSLA", "META"]
data = pd.DataFrame()

for stock in list_stock:
    stock_data = yf.download(stock, start='2015-01-01', end='2022-12-31')['Adj Close']
    data[stock] = stock_data

data

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,AAPL,MSFT,AMZN,NVDA,TSLA,META
Date,,,,,,
2015-01-02,24.435270,40.378063,15.426000,4.832578,14.620667,78.366852
2015-01-05,23.746889,40.006760,15.109500,4.750957,14.006000,77.108192
2015-01-06,23.749123,39.419556,14.764500,4.606915,14.085333,76.069290
2015-01-07,24.082134,39.920410,14.921000,4.594912,14.063333,76.069290
2015-01-08,25.007427,41.094791,15.023000,4.767760,14.041333,78.097137
...,...,...,...,...,...,...
2022-12-23,130.959946,236.195023,85.250000,151.985428,123.150002,117.914894
2022-12-27,129.142441,234.443802,83.040001,141.140747,109.099998,116.756119
2022-12-28,125.179680,232.039612,81.820000,140.291168,112.709999,115.497459


In [3]:
monthly_prices = data.resample('M').last()  # Use 'last' to get the last price of each month
monthly_returns = monthly_prices.pct_change()

# Assuming risk-free rate is 0, excess returns are the same as raw returns
excess_returns = monthly_returns

excess_returns

,AAPL,MSFT,AMZN,NVDA,TSLA,META
Date,,,,,,
2015-01-31,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-28,0.100776,0.093120,0.072293,0.153382,-0.001277,0.040311
2015-03-31,-0.031372,-0.072748,-0.021202,-0.051223,-0.071653,0.041155
2015-04-30,0.005786,0.196262,0.133513,0.060678,0.197489,-0.041961
2015-05-31,0.045339,-0.030334,0.017663,0.001461,0.109489,0.005332
...,...,...,...,...,...,...
2022-08-31,-0.031208,-0.066663,-0.060615,-0.168970,-0.072489,0.024073
2022-09-30,-0.120977,-0.109267,-0.108622,-0.195534,-0.037589,-0.167250
2022-10-31,0.109552,-0.003306,-0.093451,0.111871,-0.142168,-0.313384


# Question 1:
- Based on the sample data, compute the Markowitz portfolio weights.

# Index Weighting:
- Apple : 7.10%
- Microsoft: 6.51%
- Amazon: 3.24%
- NVIDIA: 2.84%
- Tesla : 1.87%
- Meta : 1.84%

In [4]:
weight=np.array([7.10, 6.51, 3.24, 2.84, 1.87, 1.84])
weight/100

array([0.071 , 0.0651, 0.0324, 0.0284, 0.0187, 0.0184])

In [5]:
ones=np.ones((10,1))
cov_matrix=excess_returns.cov()
cov_matrix_inverse=np.linalg.inv(cov_matrix)

In [6]:
cov_matrix_inverse*(excess_returns["AAPL"].mean())

array([[ 6.10760834e+00, -2.02368577e+00, -3.74985190e-01,
        -1.04274238e+00, -8.19829702e-01, -4.46334668e-01],
       [-2.02368577e+00,  1.24418140e+01, -3.12646698e+00,
        -1.33900931e+00, -2.36229605e-01, -9.26922122e-01],
       [-3.74985190e-01, -3.12646698e+00,  5.15307381e+00,
        -4.92595614e-01, -3.29387878e-01, -8.54776087e-01],
       [-1.04274238e+00, -1.33900931e+00, -4.92595614e-01,
         2.03701230e+00,  9.72279330e-02, -2.93131483e-02],
       [-8.19829702e-01, -2.36229605e-01, -3.29387878e-01,
         9.72279330e-02,  9.02970908e-01,  1.17496356e-02],
       [-4.46334668e-01, -9.26922122e-01, -8.54776087e-01,
        -2.93131483e-02,  1.17496356e-02,  3.10267355e+00]])

In [7]:
# As in the course, we take γmkt = 3.0271189. You said by email we could compute the γmkt with the mkt Sharpe ratio and vol or choose this value.

gamma=3.0271189
weight_mvu=1/gamma*cov_matrix_inverse@(excess_returns.mean())
data_weights=pd.DataFrame(index=list_stock)
data_weights['Mean Excess return %']=list(excess_returns.mean()*100)
data_weights['Weights Market %']=weight
data_weights['Weights MVU %']=list(weight_mvu*100)
data_weights

,Mean Excess return %,Weights Market %,Weights MVU %
AAPL,2.041489,7.10,-18.505237
MSFT,2.231980,6.51,150.187932
AMZN,2.057845,3.24,-23.694902
NVDA,4.588606,2.84,57.565286
TSLA,3.798792,1.87,16.323365
META,0.951350,1.84,-30.376249


# Question 2:
- Then, using the market-capitalization weights, obtain the CAPM-implied
expected returns.

In [8]:
gamma*cov_matrix@(weight/100)

AAPL    0.003791
MSFT    0.002760
AMZN    0.003698
NVDA    0.005255
TSLA    0.006191
META    0.002781
dtype: float64

In [9]:
data_weights['Implied CAPM excess returns %']=list((gamma*cov_matrix@(weight/100))*100)
data_weights

,Mean Excess return %,Weights Market %,Weights MVU %,Implied CAPM excess returns %
AAPL,2.041489,7.10,-18.505237,0.379140
MSFT,2.231980,6.51,150.187932,0.275973
AMZN,2.057845,3.24,-23.694902,0.369832
NVDA,4.588606,2.84,57.565286,0.525534
TSLA,3.798792,1.87,16.323365,0.619111
META,0.951350,1.84,-30.376249,0.278145


# Question 3:
- Then, specify the pick matrix P and the view vector q that captures the following views for each of the assets:
    - AAPL: its absolute excess return is expected to be 10% per year.
    - MSFT: its absolute excess return is expected to be 5% per year.
    - AMZN: no views
    - NVDA will outperform TSLA by 2% per year.
    - TSLA will underperform META by 1% per year.
- Finally, explain your choice for the matrix Ω, which captures the uncertainty about these views.

In [10]:
P=np.array([
    [1, 0, 0, 0, 0, 0],  # AAPL view
    [0, 1, 0, 0, 0, 0],  # MSFT view
    [0, 0, 0, 0, 0, 0],  # AMZN view (no view)
    [0, 0, 0, 1, -1, 0],  # NVDA vs TSLA view
    [0, 0, 0, 0, -1, 1],  # TSLA vs META view
])
q= np.array([0.10, 0.05, 0.0, 0.02, -0.01])

In [11]:
tau=1/(8*12)
omega = np.diag([P[k] @ (tau * cov_matrix) @ P[k].T for k in range(len(P))])
omega[2][2]=1e-20 # we need to modify the value 0 on the diagonal by a number close to 0 to compute the inverse of the matrix later
omega

array([[7.42559990e-05, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 4.18548416e-05, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e-20, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.72455967e-04,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        3.43055374e-04]])

Please note that in order to compute the Omega matrix we had to replace the element 2 2 on the diagonal corresponding to no view on Amazon by a very small number close to 0 because when computing the conditional expected excess return and conditional covariance matrix of excess returns, we needed to compute the inverse of the Omega matrix. However, the inverse of a diagonal matrix is the inverse of the elements on the diagonal. But the inverse of 0 doesn't exist so that's why we modified the value.

- The matrix Ω in the Black-Litterman model represents the uncertainty associated with our views. Each diagonal element corresponds to the uncertainty of the respective view. The larger the value on the diagonal, the higher the uncertainty. The values on the diagonal will make the resulting portfolio weights vary.
- We choose to use the same formula as in the course : $P_k (\tau \Sigma) P_k^T $ . The view is sensitive to the covariances associated to each asset as well as to the number of data points. This is what this formula shows. 

# Question 4
- Use these views to compute the conditional expected excess return and conditional covariance matrix of excess returns μBL and ΣBL.

In [12]:
t=1/(8*12) # nb of data points used in the estimation of returns is 8 years of monthly data so 12*8
p_transpose=P.T
t_covmatrix=np.linalg.inv(t*cov_matrix)
matrix_1=np.linalg.inv(t_covmatrix+p_transpose@np.linalg.inv(omega)@P)
matrix_2=t_covmatrix@(data_weights['Implied CAPM excess returns %']/100)+p_transpose@np.linalg.inv(omega)@q
return_bl=matrix_1@matrix_2
data_weights['Returns BL %']=return_bl*100
data_weights

,Mean Excess return %,Weights Market %,Weights MVU %,Implied CAPM excess returns %,Returns BL %
AAPL,2.041489,7.10,-18.505237,0.379140,5.649064
MSFT,2.231980,6.51,150.187932,0.275973,3.612124
AMZN,2.057845,3.24,-23.694902,0.369832,4.165101
NVDA,4.588606,2.84,57.565286,0.525534,6.443761
TSLA,3.798792,1.87,16.323365,0.619111,5.510895
META,0.951350,1.84,-30.376249,0.278145,3.348192


- Now we compute the conditional covariance matrix of excess returns

In [13]:
sigma_bl=cov_matrix+matrix_1
sigma_bl

,AAPL,MSFT,AMZN,NVDA,TSLA,META
AAPL,0.007161,0.003315,0.004209,0.006520,0.008152,0.003213
MSFT,0.003315,0.004037,0.003959,0.005112,0.004922,0.002804
AMZN,0.004209,0.003959,0.008485,0.006525,0.007192,0.004161
NVDA,0.006520,0.005112,0.006525,0.018081,0.007657,0.004410
TSLA,0.008152,0.004922,0.007192,0.007657,0.033144,0.004592
META,0.003213,0.002804,0.004161,0.004410,0.004592,0.009112


# Question 5:
- Use μBL and ΣBL to compute the mean-variance weights and compare them with the weights from the CAPM and the weights based on sample moments.

In [14]:
weight_bl=1/gamma*np.linalg.inv(sigma_bl)@return_bl
data_weights['Weights BL %']=weight_bl*100
data_weights

,Mean Excess return %,Weights Market %,Weights MVU %,Implied CAPM excess returns %,Returns BL %,Weights BL %
AAPL,2.041489,7.10,-18.505237,0.379140,5.649064,207.672773
MSFT,2.231980,6.51,150.187932,0.275973,3.612124,120.437465
AMZN,2.057845,3.24,-23.694902,0.369832,4.165101,3.206598
NVDA,4.588606,2.84,57.565286,0.525534,6.443761,12.643085
TSLA,3.798792,1.87,16.323365,0.619111,5.510895,-19.500152
META,0.951350,1.84,-30.376249,0.278145,3.348192,13.339541


- In the BL weights, we have negative weight for TSLA.

- The BL weights for AAPL and META are significantly higher compared to their market weights and sample weights.

- For the rest of stocks, BL weights are between the Market and Sample weights.